# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
# imports
import pandas as pd
import requests
import pprint
import os

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [6]:
#Citybike API notebok

import pandas as pd
import requests
import pprint


# I chose the toronto bikeshare app for this project since I personally use it and have a membership.
request_url = f'http://api.citybik.es/v2/networks/bixi-toronto'
response = requests.get(request_url, headers={'network': 'location/JSON'}).json()


# Extract the 'stations' data from the response
stations_data = response['network']['stations']

#Keeping the three columns Latitude, Longitude and number of bikes
columns_to_keep = ['name','free_bikes', 'latitude', 'longitude']

# Create a DataFrame from the 'stations' data
df1 = pd.DataFrame(stations_data, columns=columns_to_keep)

# This prints all the bike stations in toronto and shows the number of bikes, latitude and longitude.
#I used df.head() to show the data in a tabular format.

df1.head(670)


#extracting each bikes longitude and latitude
lat = df1['latitude'].tolist()
long = df1['longitude'].tolist()


In [24]:
# Define the file path for the CSV file
csv_file_city = "city_data.csv"

# Save the DataFrame to a CSV file
df1.to_csv(csv_file_city, index=False)

In [7]:
# Retreiving API key from environment variable
key = os.environ.get('foursqaure_api')


In [22]:
import json
import pandas as pd

#these are the columns we want to present in our Dataframe
desired_columns = ['name', 'location', 'distance', 'categories']
Foursquare_df = pd.DataFrame()

for n in range(len(df1)):
    name = df1.at[n, 'name']
    lat = df1.at[n, 'latitude']
    long = df1.at[n, 'longitude']
    free_bikes = df1.at[n, 'free_bikes']
    print("Station Name:", name, "lat:", lat, "long:", long, "Number of free bikes are:", free_bikes)
    
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius=1000&categories=13064%2C%2013003%2C%2013035%2C%2017114"
    headers = {
        "accept": "application/json",
        "Authorization": key  
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    # Save the Foursquare JSON data to a single file
    with open('foursquare.json', 'w') as json_file:
        json.dump(data, json_file, indent=2)

    # Define the file path for the raw json file
    json_foursquare = 'foursquare.json'  # Update the file path accordingly

    # Reading the JSON data from the file
    with open(json_foursquare, 'r') as json_file:
        data = json.load(json_file)

    POI_data = data['results']
    df3 = pd.DataFrame(POI_data)
    
    # Check if 'location' key exists in the DataFrame and apply lambda function conditionally
    df3['location'] = df3['location'].apply(lambda loc: loc['address'] if 'address' in loc else None)
    df3['categories'] = df3['categories'].apply(lambda x: x[0]['name'] if x else '')
    df3['bike_station_name'] = name
    df3['Latitude'] = lat
    df3['Longitude'] = long
    df3['bike_station_coordinates'] = df3['Latitude'].astype(str) + ', ' + df3['Longitude'].astype(str)

    selected_columns = df3[desired_columns]
    display(selected_columns.head())
    Foursquare_df = pd.concat([Foursquare_df, df3])


Station Name: Queen St E / Woodward Ave lat: 43.665269 long: -79.319796 Number of free bikes are: 18


,name,location,distance,categories
0,The Sidekick,1374 Queen St E,440,Coffee Shop
1,Hasting Snack Bar,5 Hastings Ave,759,Lounge
2,Lake Inez,1471 Gerrard St E,809,Bar
3,Godspeed Brewery,242 Coxwell Ave,804,Bar
4,O Sushi,6 Coxwell Ave,300,Pizzeria


Station Name: Primrose Ave / Davenport Rd lat: 43.67142 long: -79.445947 Number of free bikes are: 0


,name,location,distance,categories
0,Balzac's Coffee Roasters,31 Powerhouse St,284,Coffee Shop
1,The Greater Good Bar,229 Geary Ave,581,Bar
2,Hale Coffee,300 Campbell St,729,Coffee Shop
3,Marcello's Pizzeria,1163 Saint Clair Ave W,777,Pizzeria
4,Pizza e Pazzi,1182 Saint Clair Ave W,733,Pizzeria


Station Name: Queen St. E / Rhodes Ave. lat: 43.666224 long: -79.317693 Number of free bikes are: 22


...

,name,location,distance,categories
0,Starbucks,50 Musgrave St,697,Coffee Shop
1,Pizza Pizza,2795 Danforth Ave,634,Pizzeria
2,The Daily Perk,Victoria Park Ave,284,Coffee Shop
3,Subhan Halal Pizza,2970 Danforth Ave,428,Pizzeria
4,Country Style,2929 Danforth Ave,519,Coffee Shop


Station Name: Lower Jarvis / Queens Quay E lat: 43.64422 long: -79.36927 Number of free bikes are: 12


,name,location,distance,categories
0,Neo Coffee Bar,161 Frederick St,720,Café
1,Mos Mos,20 Bay St,725,Coffee Shop
2,Rooster Coffee House,343 King St E,902,Coffee Shop
3,Arvo Coffee,17 Gristmill Lane,896,Café
4,Fahrenheit Coffee,120 Lombard St,948,Café


Station Name: Summerhill Ave / Maclennan Ave lat: 43.685924 long: -79.376304 Number of free bikes are: 0


KeyError: 'location'

In [18]:
#foursqaure function to extract the data
def extract_json(json):
    POI_data = json['results']
    return POI_data

In [19]:
#calling the foursqaure function
extract_json(data)

[{'fsq_id': '4bdca8662a3a0f4757d4b3b6',
  'categories': [{'id': 13022,
    'name': 'Sports Bar',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_',
     'suffix': '.png'}},
   {'id': 13065,
    'name': 'Restaurant',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 980,
  'geocodes': {'main': {'latitude': 43.719493, 'longitude': -79.249742},
   'roof': {'latitude': 43.719493, 'longitude': -79.249742}},
  'link': '/v3/places/4bdca8662a3a0f4757d4b3b6',
  'location': {'address': '3676 Stclair Ave E',
   'country': 'CA',
   'formatted_address': '3676 Stclair Ave E, Toronto ON M1M 1T2',
   'locality': 'Toronto',
   'postcode': 'M1M 1T2',
   'region': 'ON'},
  'name': 'Working Dog Saloon',
  'related_places': {},
  'timezone': 'America/Toronto'},
 {'fsq_id': '4c85a8c2ee6fef3b9b123e5c',
  'categories': [{'id': 13064,
    'name': 'Pizzeria',
    'icon': {'prefix': 'https://ss

In [23]:
#Saving the Foursquare API DataFrame into a CSV file so I can work on it and not have to waste my API calls
# Save the Foursquare_df DataFrame to a CSV file
# Define the file path for the CSV file
csv_file_path = "foursquare_data.csv"

# Save the DataFrame to a CSV file
Foursquare_df.to_csv(csv_file_path, index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [15]:
import os
import pandas as pd 
yelp_key = os.environ.get('yelp_key')


Yelp_columns = ['name', 'distance', 'location', 'POI_category']

yelp_dataframe = pd.DataFrame()

for n in range(len(df1)):
    name = df1.at[n, 'name']
    lat = df1.at[n, 'latitude']
    long = df1.at[n, 'longitude']
    free_bikes = df1.at[n, 'free_bikes']
    print("Station Name:", name, "lat:", lat, "long:", long, "Number of free bikes are:", free_bikes)
    
    url = f"https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000&categories=breakfast_brunch%2Ccoffee%2Cgrocery&sort_by=best_match&limit=20"

    yelp_api = yelp_key
    headers = {
        "accept": "application/json",
        "Authorization": f"bearer {yelp_api}"
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    
    #Saving the Raw Json file so we can call on it and use it instead of wasting API calls:
    # Save all the YELP JSON data to a single file
    with open('yelp.json', 'w') as json_file:
        json.dump(data, json_file, indent=2)
    
    
    # Extracting relevant information from the response data
    yelp_POI = data['businesses']
    yelp_df = pd.DataFrame(yelp_POI)
    yelp_df['location'] = yelp_df['location'].apply(lambda loc: loc['address1'])
    yelp_df['bike_station_name'] = name
    yelp_df['Latitude'] = lat
    yelp_df['Longitude'] = long
    yelp_df['POI_category'] = yelp_df['categories'].apply(lambda x: x[0]['title'] if len(x) > 0 else "")
    yelp_df['bike_station_coordinates'] = yelp_df['Latitude'].astype(str) + ', ' + yelp_df['Longitude'].astype(str)
    
    Yelp_selected_columns = yelp_df[Yelp_columns]
    display(Yelp_selected_columns.head())
    yelp_dataframe = pd.concat([yelp_dataframe, yelp_df])
    

            
        

Station Name: Queen St E / Woodward Ave lat: 43.665269 long: -79.319796 Number of free bikes are: 18


,name,distance,location,POI_category
0,McDonald's,32.416737,29-31 Woodward Avenue,Fast Food
1,Yaya Greek Bistro,994.967613,1186 Queen Street E,Breakfast & Brunch
2,Lazy Daisy's Cafe,828.662056,1515 Gerrard Street E,Cafes
3,Starbucks,930.683769,4 - 23 Leslie Street,Coffee & Tea
4,Pilot Coffee Roasters,1251.969414,50 Wagstaff Drive,Coffee & Tea


Station Name: Primrose Ave / Davenport Rd lat: 43.67142 long: -79.445947 Number of free bikes are: 0


,name,distance,location,POI_category
0,Balzac's Coffee Roasters,261.708843,29 Powerhouse Street,Coffee Roasteries
1,Lox + Schmear,1060.466420,1030 St Clair W,Bagels
2,Mofer Coffee,1016.047026,1040 St Clair Avenue W,Coffee & Tea Supplies
3,Wallace Espresso,1009.826494,188 Wallace Avenue,Coffee & Tea
4,Sunnyside Grill,717.529331,1218 Saint Clair Avenue W,Breakfast & Brunch


Station Name: Queen St. E / Rhodes Ave. lat: 43.666224 long: -79.317693 Number of free bikes are: 22


...

,name,distance,location,POI_category
0,Moore's Valu-Mart,977.838799,123 Guildwood Parkway,Grocery
1,Tim Hortons,965.587744,91 Guildwood Pky,Coffee & Tea


Station Name: Chorley Park - SMART lat: 43.6856 long: -79.3718 Number of free bikes are: 9


,name,distance,location,POI_category
0,Summerhill Market,304.711839,446 Summerhill Avenue,Grocery
1,Rosedale's Finest,385.628548,408 Summerhill Avenue,Grocery


Station Name: Cherry Beach lat: 43.637764 long: -79.345359 Number of free bikes are: 45


KeyError: 'location'

In [ ]:
#yelp function to extract the data
def extract_json(json):
    POI_data = json['businesses']
    return POI_data

In [ ]:
#calling the yelp function
extract_json(data)

In [16]:
#Saving the YELP API DataFrame into a CSV file so I can work on it and not have to waste my API calls
# Save the YELP DataFrame to a CSV file
# Define the file path for the CSV file
csv_file_yelp = "yelp.csv"

# Save the DataFrame to a CSV file
yelp_dataframe.to_csv(csv_file_yelp, index=False)

print("DataFrame saved to yelp.csv successfully!")

DataFrame saved to yelp.csv successfully!


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [ ]:
'''Yelp gives you more information about the POI since the website allows users to rate the POI. It provides the ratings and the number 
of ratings. This is much more convenient since it can show you the most top rated POI categories we have chosen next to a bike station in toronto'''

In [ ]:
#For data audits, I checked for null values and outliers for the Foursqaure Dataframe.
# Check for null values in the DataFrame
null_values = Foursquare_df.isnull()

# Count the number of null values in each column
null_counts = Foursquare_df.isnull().sum()

# Display the DataFrame containing boolean values for null check
print("Null Values Check:")
print(null_values)

# Display the count of null values in each column
print("\nNull Value Counts:")
print(null_counts)


# Get summary statistics of numerical columns will show any outliers we need to be aware of
summary_stats_foursquare = Foursquare_df.describe()
print(summary_stats_foursquare)

In [ ]:
#For data audits, I checked for null values and outliers for the Yelp Dataframe.
# Check for null values in the DataFrame
null_values = yelp_dataframe.isnull()

# Count the number of null values in each column
null_counts = yelp_dataframe.isnull().sum()

# Display the DataFrame containing boolean values for null check
print("Null Values Check:")
print(null_values)

# Display the count of null values in each column
print("\nNull Value Counts:")
print(null_counts)

# Get summary statistics of numerical columns will show any outliers we need to be aware of
summary_stats_yelp = yelp_dataframe.describe()
print(summary_stats_yelp)